In [9]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob

def findFiles(path): return glob.glob(path)

print(findFiles('data/doc/*.txt'))

import unicodedata
import string


all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

#將Unicode轉ASCII
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

#print(unicodeToAscii('Ślusàrski'))

# 構建category_lines字典，每種語言的名稱列表
category_lines = {}
all_categories = []

# 讀取文件並分行
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [line for line in lines]

for filename in findFiles('data/doc/*.txt'):
    category = filename.split('/')[-1].split('.')[0]
    #18種類別
    all_categories.append(category)
    lines = readLines(filename)
    #category_lines 類別：名字List
    category_lines[category] = lines

n_categories = len(all_categories)

['data/doc/china.txt', 'data/doc/social.txt']


In [10]:
print("all_letters: "+str(len(all_letters)))
print("n_categories: "+str(n_categories))
#print("category_lines:"+str(category_lines['雙亡'][:5]))

all_letters: 57
n_categories: 2


In [11]:
import torch

# 建立函式方便尋找all_letters的Index像是"a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# 將一個字母轉成Tensor顯示
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# 建立one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(letterToTensor('Z'))

print(lineToTensor('Jones').size())

tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]])
torch.Size([5, 1, 57])


In [12]:
import torch.nn as nn

 
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

In [13]:
input = letterToTensor('A')
hidden =torch.zeros(1, n_hidden)

output, next_hidden = rnn(input, hidden)
print(output)

tensor([[-0.7584, -0.6319]])


In [14]:
input = lineToTensor('Albert')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input[0], hidden)
print(output)

tensor([[-0.7584, -0.6319]])


In [15]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i

print(categoryFromOutput(output))

('social', 1)


In [16]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)

category = china / line = 的
category = social / line = 陳宏育
category = social / line = 與
category = china / line = 大陸
category = china / line = 6
category = china / line = 葉青林
category = china / line = 大陸
category = china / line = 採訪
category = social / line = 27
category = social / line = 墜谷


In [17]:
criterion = nn.NLLLoss()

In [18]:
learning_rate = 0.005

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item()

In [19]:
import time
import math

n_iters = 100000
print_every = 5000
plot_every = 1000



# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    # Print iter number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

5000 5% (0m 2s) 0.6260 日 / social ✓
10000 10% (0m 4s) 0.7040 報導 / china ✗ (social)
15000 15% (0m 6s) 0.6364 來台 / china ✓
20000 20% (0m 8s) 0.8700 據悉 / china ✗ (social)
25000 25% (0m 10s) 0.8097 東南衛視 / social ✗ (china)
30000 30% (0m 12s) 0.7333 6 / social ✗ (china)
35000 35% (0m 13s) 0.5928 申請 / china ✓
40000 40% (0m 15s) 0.7182 據悉 / china ✗ (social)
45000 45% (0m 17s) 0.6030 大陸 / china ✓
50000 50% (0m 19s) 0.7721 報導 / china ✗ (social)
55000 55% (0m 21s) 0.6780 27 / social ✓
60000 60% (0m 23s) 0.6827 日 / social ✓
65000 65% (0m 25s) 1.1139 東南衛視 / social ✗ (china)
70000 70% (0m 26s) 0.7490 陸委會 / social ✗ (china)
75000 75% (0m 28s) 0.7538 報導 / china ✗ (social)
80000 80% (0m 30s) 0.7729 范錦美 / china ✗ (social)
85000 85% (0m 32s) 0.6755 昨天 / china ✓
90000 90% (0m 34s) 0.5514 在 / social ✓
95000 95% (0m 36s) 0.6980 27 / china ✗ (social)
100000 100% (0m 37s) 0.6849 王錦義 / social ✓


In [29]:
# 回傳輸出
def evaluate(line_tensor):
    hidden = rnn.initHidden()
    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)
    return output


def predict(input_line, n_predictions=1):
    print('\n> %s' % input_line)
    with torch.no_grad():
        output = evaluate(lineToTensor(input_line))

        # 獲得N個類別
        topv, topi = output.topk(n_predictions, 1, True)
        predictions = []

        for i in range(n_predictions):
            value = topv[0][i].item()
            category_index = topi[0][i].item()
            print(all_categories[category_index])
            #print('(%.2f) %s' % (value, all_categories[category_index]))
            predictions.append([value, all_categories[category_index]])

predict('公路')


> 公路
social
